In [1]:
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras import activations

In [5]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [1064]:
data_raw = pd.read_csv(r'D:\HackerEarth\Attrition\Dataset\Train.csv')

In [1065]:
test_raw = pd.read_csv(r'D:\HackerEarth\Attrition\Dataset\Test.csv')

In [1066]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 24 columns):
Employee_ID                  7000 non-null object
Gender                       7000 non-null object
Age                          6588 non-null float64
Education_Level              7000 non-null int64
Relationship_Status          7000 non-null object
Hometown                     7000 non-null object
Unit                         7000 non-null object
Decision_skill_possess       7000 non-null object
Time_of_service              6856 non-null float64
Time_since_promotion         7000 non-null int64
growth_rate                  7000 non-null int64
Travel_Rate                  7000 non-null int64
Post_Level                   7000 non-null int64
Pay_Scale                    6991 non-null float64
Compensation_and_Benefits    7000 non-null object
Work_Life_balance            6989 non-null float64
VAR1                         7000 non-null int64
VAR2                         6423 non-null fl

In [1067]:
# data_raw['Attrition_rate'] = np.where(data_raw.Attrition_rate>=0.5,0.5,data_raw.Attrition_rate)
# data_raw.shape
# data_raw = data_raw[data_raw.Attrition_rate<=0.4]
data_raw.shape

(7000, 24)

In [1068]:
total = data_raw.isnull().sum().sort_values(ascending=False)
percent_1 = data_raw.isnull().sum()/data_raw.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data[missing_data.Total!=0]

,Total,%
VAR4,656,9.4
VAR2,577,8.2
Age,412,5.9
Time_of_service,144,2.1
Work_Life_balance,11,0.2
Pay_Scale,9,0.1


In [1069]:
total = test_raw.isnull().sum().sort_values(ascending=False)
percent_1 = test_raw.isnull().sum()/test_raw.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data[missing_data.Total!=0]

,Total,%
VAR4,298,9.9
VAR2,217,7.2
Age,161,5.4
Time_of_service,52,1.7
Work_Life_balance,5,0.2
Pay_Scale,3,0.1


In [1070]:
corr = data_raw.corr()
corr

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
Age,1.000000,-0.005770,0.918853,0.145760,0.001270,0.012266,0.002440,0.012966,0.003073,0.005696,0.005406,0.007569,0.001803,0.000015,-0.000832,0.008746,-0.015498
Education_Level,-0.005770,1.000000,-0.012793,-0.002077,0.002991,-0.017690,0.009986,0.006834,-0.015262,-0.002185,-0.010312,-0.009689,0.004500,0.005444,0.009195,0.012909,-0.008143
Time_of_service,0.918853,-0.012793,1.000000,0.137014,-0.003486,0.004843,-0.002286,0.020347,-0.002557,0.004757,0.003858,0.006863,-0.006024,0.003194,0.007618,0.007034,-0.016447
Time_since_promotion,0.145760,-0.002077,0.137014,1.000000,0.014147,-0.012061,-0.008851,-0.008901,-0.007423,-0.002873,0.004211,-0.002783,-0.003724,0.000563,0.017264,0.005173,0.013880
growth_rate,0.001270,0.002991,-0.003486,0.014147,1.000000,0.008460,-0.006887,-0.004145,-0.018324,0.001351,0.010883,0.010991,0.021336,-0.015122,-0.005860,-0.017993,0.014247
Travel_Rate,0.012266,-0.017690,0.004843,-0.012061,0.008460,1.000000,0.021299,-0.011689,0.004132,0.023898,0.008374,-0.009019,-0.005933,-0.003866,-0.012280,-0.006271,-0.012608
Post_Level,0.002440,0.009986,-0.002286,-0.008851,-0.006887,0.021299,1.000000,0.022389,-0.014473,-0.008458,-0.007043,0.010387,-0.012778,0.013008,0.002765,-0.003397,0.016402
Pay_Scale,0.012966,0.006834,0.020347,-0.008901,-0.004145,-0.011689,0.022389,1.000000,-0.022742,0.011354,0.006296,0.003380,-0.010286,0.018080,-0.012318,0.002381,-0.015236
Work_Life_balance,0.003073,-0.015262,-0.002557,-0.007423,-0.018324,0.004132,-0.014473,-0.022742,1.000000,-0.022030,-0.001492,0.004476,-0.026753,0.022196,0.003871,0.024534,0.020746
VAR1,0.005696,-0.002185,0.004757,-0.002873,0.001351,0.023898,-0.008458,0.011354,-0.022030,1.000000,0.002260,-0.003318,0.002112,0.007904,-0.002771,0.009355,-0.008073


In [1071]:
data_mod1 = data_raw.drop(['Employee_ID','Attrition_rate'],axis = 1).copy()
test_mod1 = test_raw.drop(['Employee_ID'],axis = 1).copy()

In [1072]:
test_mod1.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,F,32.0,5,Single,Springfield,R&D,Conceptual,7.0,4,30,...,4.0,type2,1.0,3,-0.9612,-0.4537,2.0,1,8,4
1,M,65.0,2,Single,Lebanon,IT,Directive,41.0,2,72,...,1.0,type2,1.0,4,-0.9612,0.7075,1.0,2,8,2
2,M,52.0,3,Married,Springfield,Sales,Directive,21.0,3,25,...,8.0,type3,1.0,4,-0.1048,0.7075,2.0,1,9,3
3,M,50.0,5,Single,Washington,Marketing,Analytical,11.0,4,28,...,2.0,type0,4.0,3,-0.1048,0.7075,2.0,2,8,3
4,F,44.0,3,Married,Franklin,R&D,Conceptual,12.0,4,47,...,2.0,type2,4.0,4,1.6081,0.7075,2.0,2,7,4


In [1073]:
Y_full = data_raw['Attrition_rate']

In [1074]:
# do missing value imputation with 'mode' for 2 variables - 'work life bal', 'payscale'; 
# because the count of missing valus is very low
imp = SimpleImputer(strategy='most_frequent',missing_values=np.nan)

imp.fit(data_raw[['Work_Life_balance']])
data_mod1['Work_Life_balance'] = imp.transform(data_raw[['Work_Life_balance']])
test_mod1['Work_Life_balance'] = imp.transform(test_raw[['Work_Life_balance']])

imp.fit(data_raw[['Pay_Scale']])
data_mod1['Pay_Scale'] = imp.transform(data_raw[['Pay_Scale']])
test_mod1['Pay_Scale'] = imp.transform(test_raw[['Pay_Scale']])

In [1075]:
# do outlier treatment
# sns.boxplot(x=data_mod1['Education_Level'])


# data_mod1.boxplot()

for col in data_mod1.columns:
    print('-' * 40 + col + '-' * 40 , end=' - ')
    if col != 'Employee_ID':
        display(data_mod1[col].value_counts())
        
# looks like no outliers based on the value counts

----------------------------------------Gender---------------------------------------- - 

F    4114
M    2886
Name: Gender, dtype: int64

----------------------------------------Age---------------------------------------- - 

22.0    235
25.0    235
27.0    231
31.0    218
54.0    212
60.0    211
26.0    210
37.0    208
35.0    205
52.0    199
28.0    194
47.0    193
42.0    193
30.0    191
32.0    189
24.0    185
45.0    184
21.0    171
65.0    152
19.0    151
23.0    148
59.0    136
20.0    122
58.0    122
29.0    115
36.0    106
33.0    104
51.0    103
50.0    101
34.0     99
46.0     98
49.0     97
64.0     95
55.0     94
56.0     94
43.0     93
41.0     92
57.0     92
38.0     89
53.0     88
44.0     85
39.0     83
40.0     81
48.0     79
63.0     73
62.0     69
61.0     63
Name: Age, dtype: int64

----------------------------------------Education_Level---------------------------------------- - 

...

3    2758
2    2085
4    1450
1     485
5     222
Name: VAR5, dtype: int64

----------------------------------------VAR6---------------------------------------- - 

8    2661
7    1734
6    1174
5     910
9     521
Name: VAR6, dtype: int64

----------------------------------------VAR7---------------------------------------- - 

3    3926
4    1320
5     840
2     627
1     287
Name: VAR7, dtype: int64

In [1076]:
# convert categorical variables to dummy
# pd.get_dummies(data_mod1[['Gender','Relationship_Status','Hometown','Unit','Decision_skill_possess',]],drop_first=True)
non_cat_var = ['Age','Time_of_service','Time_since_promotion','growth_rate']

dummy_var_df = pd.DataFrame()
for col in data_mod1.columns:
    if col not in non_cat_var:
        dummy_var_df = pd.concat([dummy_var_df,pd.get_dummies(data_mod1[col],prefix=col,drop_first=True)],
                                 axis=1,ignore_index=False)
        

dummy_var_df.head()

,Gender_M,Education_Level_2,Education_Level_3,Education_Level_4,Education_Level_5,Relationship_Status_Single,Hometown_Franklin,Hometown_Lebanon,Hometown_Springfield,Hometown_Washington,...,VAR5_4,VAR5_5,VAR6_6,VAR6_7,VAR6_8,VAR6_9,VAR7_2,VAR7_3,VAR7_4,VAR7_5
0,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0


In [1077]:
dummy_var_df_test = pd.DataFrame()
for col in test_mod1.columns:
    if col not in non_cat_var:
        dummy_var_df_test = pd.concat([dummy_var_df_test,pd.get_dummies(test_mod1[col],prefix=col,drop_first=True)],
                                 axis=1,ignore_index=False)
        
dummy_var_df_test.head()

,Gender_M,Education_Level_2,Education_Level_3,Education_Level_4,Education_Level_5,Relationship_Status_Single,Hometown_Franklin,Hometown_Lebanon,Hometown_Springfield,Hometown_Washington,...,VAR5_4,VAR5_5,VAR6_6,VAR6_7,VAR6_8,VAR6_9,VAR7_2,VAR7_3,VAR7_4,VAR7_5
0,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
2,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
3,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [1078]:
# concatenate dummy variable DF to numeric variable columns
data_mod2 = pd.concat([data_mod1[non_cat_var],dummy_var_df],axis = 1,ignore_index=False)
# data_mod2.head()

test_mod2 = pd.concat([test_mod1[non_cat_var],dummy_var_df_test],axis = 1,ignore_index=False)
test_mod2.head()

,Age,Time_of_service,Time_since_promotion,growth_rate,Gender_M,Education_Level_2,Education_Level_3,Education_Level_4,Education_Level_5,Relationship_Status_Single,...,VAR5_4,VAR5_5,VAR6_6,VAR6_7,VAR6_8,VAR6_9,VAR7_2,VAR7_3,VAR7_4,VAR7_5
0,32.0,7.0,4,30,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,1,0
1,65.0,41.0,2,72,1,1,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
2,52.0,21.0,3,25,1,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,50.0,11.0,4,28,1,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0
4,44.0,12.0,4,47,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [1079]:
#do this after converting all variables to categorical. Strings will throw error
knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")
# imp = knn_imputer.fit_transform(data_mod2.copy())
knn_imputer.fit(data_mod2.copy())
imp = knn_imputer.transform(data_mod2.copy())
data_mod3 = pd.DataFrame(imp)

data_mod3.columns = data_mod2.columns
data_mod3.head()

,Age,Time_of_service,Time_since_promotion,growth_rate,Gender_M,Education_Level_2,Education_Level_3,Education_Level_4,Education_Level_5,Relationship_Status_Single,...,VAR5_4,VAR5_5,VAR6_6,VAR6_7,VAR6_8,VAR6_9,VAR7_2,VAR7_3,VAR7_4,VAR7_5
0,42.0,4.0,4.0,33.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,24.0,5.0,4.0,36.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,58.0,27.0,3.0,51.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,26.0,4.0,3.0,56.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,31.0,5.0,4.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [1080]:
imp = knn_imputer.transform(test_mod2.copy())
test_mod3 = pd.DataFrame(imp)

test_mod3.columns = test_mod2.columns

test_mod3.head()

,Age,Time_of_service,Time_since_promotion,growth_rate,Gender_M,Education_Level_2,Education_Level_3,Education_Level_4,Education_Level_5,Relationship_Status_Single,...,VAR5_4,VAR5_5,VAR6_6,VAR6_7,VAR6_8,VAR6_9,VAR7_2,VAR7_3,VAR7_4,VAR7_5
0,32.0,7.0,4.0,30.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,65.0,41.0,2.0,72.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,52.0,21.0,3.0,25.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,50.0,11.0,4.0,28.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,44.0,12.0,4.0,47.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [1081]:
data_mod4 = data_mod3.copy()
test_mod4 = test_mod3.copy()
data_mod4['Time_of_service'] = np.digitize(data_mod4.Time_of_service,[5,10,15,20,25,30])+1
test_mod4['Time_of_service'] = np.digitize(test_mod4.Time_of_service,[5,10,15,20,25,30])+1
data_mod4 = pd.concat([data_mod4.drop(['Time_of_service'],axis=1),
        pd.get_dummies(data_mod4['Time_of_service'],prefix = 'Time_of_service',drop_first=True)],axis=1,ignore_index=False)
test_mod4 = pd.concat([test_mod4.drop(['Time_of_service'],axis=1),
        pd.get_dummies(test_mod4['Time_of_service'],prefix = 'Time_of_service',drop_first=True)],axis=1,ignore_index=False)

In [1082]:
# data_mod4.columns

In [1083]:
# segmentation analysis
# data_mod4[['Age','Attrition_rate']].groupby('Age').mean()

In [1084]:
# df=data_mod4.copy()
# colname = 'growth_rate'
# df[colname] = np.digitize(data_mod4[colname],bins=[30])
# df[[colname,'Attrition_rate']].groupby(colname).mean()

In [1085]:
# split to train, dev sets
X_train_orig, X_dev_orig, Y_train, Y_dev = train_test_split(data_mod4,Y_full,test_size=0.3,random_state=1)
print('Training length: ' + str(X_train_orig.shape[0]))
print('Dev length: '+ str(X_dev_orig.shape[0]))

Training length: 4900
Dev length: 2100


In [1086]:
# perform mean normalization for numerical variables
non_cat_var = ['Age','Time_since_promotion','growth_rate']
scaler = preprocessing.StandardScaler().fit(X_train_orig[non_cat_var])

In [1087]:
X_train = X_train_orig.copy()
X_dev = X_dev_orig.copy()
X_test = test_mod4.copy()
X_train[non_cat_var] = scaler.transform(X_train_orig[non_cat_var])
X_dev[non_cat_var] = scaler.transform(X_dev_orig[non_cat_var])
X_test[non_cat_var] = scaler.transform(test_mod4[non_cat_var])

print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)

(4900, 82)
(2100, 82)
(3000, 82)


In [1088]:
# reg = LinearRegression().fit(X_train, Y_train)
# print(reg.score(X_train,Y_train))
# print(reg.score(X_dev,Y_dev))

In [1089]:
def train_and_evaluate(X_train, Y_train, X_dev, Y_dev, max_leaf=500):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf, random_state=0)
    ## Fitting model
    model.fit(X_train, Y_train)
    ## Training Loss
    train_preds = model.predict(X_train)
    train_score = 100*(1-np.sqrt(mean_squared_error(train_preds, Y_train)))
    predictions = model.predict(X_dev)
    valid_score = 100*(1-np.sqrt(mean_squared_error(predictions, Y_dev)))
    print("Base Line model performance \n \t \t----------------------------- \n \t \
                 Train score {:,.4f} \t Valid score {:,.4f}".format(train_score, valid_score))
    return model

In [1090]:
# model = train_and_evaluate(X_train, Y_train, X_dev, Y_dev, max_leaf=10)

In [1091]:
# test_predictions = model.predict(X_test)
# test_predictions.shape

In [1092]:
colsToExclude = ['Age']
# colsToExclude = ['Age','Hometown_Franklin','Hometown_Lebanon','Hometown_Springfield','Hometown_Washington',
#                  'Travel_Rate_1', 'Travel_Rate_2','Gender_M','Relationship_Status_Single','VAR1_2', 'VAR1_3',
#        'VAR1_4', 'VAR1_5','VAR2_-0.9612', 'VAR2_-0.1048', 'VAR2_0.7516', 'VAR2_1.6081','VAR4_2.0', 'VAR4_3.0',
#                'VAR6_6', 'VAR6_7', 'VAR6_8', 'VAR6_9','VAR7_2', 'VAR7_3',
#        'VAR7_4', 'VAR7_5' ]
# colsToExclude = []
X_train_new = X_train.drop(colsToExclude,axis=1)
X_dev_new = X_dev.drop(colsToExclude,axis=1)
X_test_new = X_test.drop(colsToExclude,axis=1)

In [1151]:
# create a kickass finely tuned neural net!!!
# GRADED FUNCTION: HappyModel

def NNModel(input_shape,lamda,inter_act = 'relu',inter_act2 ='relu', output_act='sigmoid',drop=0.4):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset
        (height, width, channels) as a tuple.  
        Note that this does not include the 'batch' as a dimension.
        If you have a batch like 'X_train', 
        then you can provide the input_shape using
        X_train.shape[1:]


    Returns:
    model -- a Model() instance in Keras
    """
    
    ### START CODE HERE ###
    # Feel free to use the suggested outline in the text above to get started, and run through the whole
    # exercise (including the later portions of this notebook) once. The come back also try out other
    # network architectures as well. 
    
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    X = Dense(80, activation=inter_act, name='fc1',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X_input)
    X = Dropout(drop)(X)
    X = Dense(80, activation=inter_act, name='fc2',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(60, activation=inter_act, name='fc3',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(60, activation=inter_act, name='fc4',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(60, activation=inter_act, name='fc5',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc6',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc7',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc8',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc9',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc10',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc11',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
    X = Dropout(drop)(X)
    X = Dense(40, activation=inter_act, name='fc12',kernel_regularizer=tf.keras.regularizers.l2(lamda))(X)
#     X = Dropout(drop)(X)
    X = Dense(1, activation=output_act, name='output')(X)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='NNModel')
    
    ### END CODE HERE ###
    
    return model

In [1173]:
# tf.compat.v1.reset_default_graph()
nnModel = NNModel(X_train_new.shape[1:],lamda=0.0002,inter_act='relu',inter_act2 = 'selu',drop=0.0,output_act='sigmoid')
#lambda 0.005, drop 0, layers 60,10
# 0.003,0.1 60,30
# 0, 0.3, 60, 30
#0, 0.5,60,60
#0,0.5,60,60,20
#0,0.5,60,60,80
#0.0006,0.2,60,60,80

# nnModel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# opt = tf.keras.optimizers.SGD(learning_rate=0.1)
opt = tf.keras.optimizers.Adam()
nnModel.compile(loss='mean_squared_error',optimizer=opt,metrics=[tf.keras.metrics.RootMeanSquaredError()])


nnModel.fit(x=X_train_new,y=Y_train,epochs=300,batch_size=X_train_new.shape[0]) #81.24
# nnModel.fit(x=X_train_new,y=Y_train,epochs=10,batch_size=32)

Epoch 1/300
1/1 [==============================] - 0s 0s/step - loss: 0.2567 - root_mean_squared_error: 0.3598
Epoch 2/300
1/1 [==============================] - 0s 4ms/step - loss: 0.2508 - root_mean_squared_error: 0.3515
Epoch 3/300
1/1 [==============================] - 0s 0s/step - loss: 0.2448 - root_mean_squared_error: 0.3430
Epoch 4/300
1/1 [==============================] - 0s 4ms/step - loss: 0.2390 - root_mean_squared_error: 0.3344
Epoch 5/300
1/1 [==============================] - 0s 4ms/step - loss: 0.2336 - root_mean_squared_error: 0.3262
Epoch 6/300
1/1 [==============================] - 0s 4ms/step - loss: 0.2284 - root_mean_squared_error: 0.3182
Epoch 7/300
1/1 [==============================] - 0s 4ms/step - loss: 0.2236 - root_mean_squared_error: 0.3105
Epoch 8/300
1/1 [==============================] - 0s 0s/step - loss: 0.2191 - root_mean_squared_error: 0.3032
Epoch 9/300
1/1 [==============================] - 0s 0s/step - loss: 0.2149 - root_mean_squared_error: 0.2

1/1 [==============================] - 0s 4ms/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 146/300
1/1 [==============================] - 0s 0s/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 147/300
1/1 [==============================] - 0s 0s/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 148/300
1/1 [==============================] - 0s 0s/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 149/300
1/1 [==============================] - 0s 0s/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 150/300
1/1 [==============================] - 0s 0s/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 151/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 152/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1618 - root_mean_squared_error: 0.1898
Epoch 153/300
1/1 [==============================] - 0s 0s/step - loss: 0.1618 - root_mean_squared_error: 0

1/1 [==============================] - 0s 0s/step - loss: 0.1604 - root_mean_squared_error: 0.1897
Epoch 290/300
1/1 [==============================] - 0s 8ms/step - loss: 0.1604 - root_mean_squared_error: 0.1897
Epoch 291/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1604 - root_mean_squared_error: 0.1897
Epoch 292/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1604 - root_mean_squared_error: 0.1897
Epoch 293/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1603 - root_mean_squared_error: 0.1897
Epoch 294/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1603 - root_mean_squared_error: 0.1897
Epoch 295/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1603 - root_mean_squared_error: 0.1897
Epoch 296/300
1/1 [==============================] - 0s 0s/step - loss: 0.1603 - root_mean_squared_error: 0.1897
Epoch 297/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1603 - root_mean_squared_erro

In [1174]:
# nnModel.fit(x=X_train_new,y=Y_train,epochs=50,batch_size=X_train_new.shape[0])
# nnModel.fit(x=X_train_new,y=Y_train,epochs=10,batch_size=32)

In [1176]:
trainPreds = nnModel.evaluate(x=X_train_new,y=Y_train)
print()
# print ("Loss = " + str(preds[0]))
print ("Train Score = " + str(100*(1-trainPreds[1])))
testPreds = nnModel.evaluate(x=X_dev_new,y=Y_dev)
### END CODE HERE ###
print()
# print ("Loss = " + str(preds[0]))
print ("Test Score = " + str(100*(1-testPreds[1])))

154/154 [==============================] - 0s 1ms/step - loss: 0.1603 - root_mean_squared_error: 0.1897

Train Score = 81.0269758105278
66/66 [==============================] - 0s 1ms/step - loss: 0.1553 - root_mean_squared_error: 0.1762

Test Score = 82.3836401104927


In [1097]:
# pred_df = X_train.copy()
# pred_df['Predicted'] = nnModel.predict(X_train)
# pred_df.to_csv(r'D:\HackerEarth\Attrition\Dataset\pred.csv')

In [1098]:
# submission = pd.concat([test_raw.Employee_ID,pd.DataFrame(nnModel.predict(X_test_new))],axis=1)
# submission.columns = ['Employee_ID','Attrition_rate']
# submission.head()
# submission.to_csv(r'D:\HackerEarth\Attrition\Dataset\newstuff1.csv',index=None)